# Carotid Dataset Creation

In this notebook we will create a dataset of $256\times256$ carotid slices that will be used to train a single image super-resolution convolutional neural network.

In [1]:
import numpy as np
import nibabel as nib
import os
from scipy.misc import imsave
from keras.preprocessing.image import ImageDataGenerator

/home/roberto/Documents/virtualenv/venv04/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
verbose = 1
H1,W1 = 256,256 # Desired resolution

nifti_path = "/media/roberto/DATA/GDrive/MR-Data/Carotid/Original-nifti"
nifti_list = [os.path.join(nifti_path,f) for f in os.listdir(nifti_path) if not f.startswith("Kmd")\
              and (not f.startswith("Djm")) and f.endswith(".nii.gz")]

out_path = "/media/roberto/DATA/ML101/Data/carotid_slices"

nsamples = 50000 # Number of carotid MR slices
nslices = nsamples/len(nifti_list) + 1

if verbose:
    print "Number of nifti files: ", len(nifti_list)
    print "Sampling rate per file: ", nslices

image_datagen = ImageDataGenerator(rotation_range=20,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.2,
                                   zoom_range=0.15,
                                   horizontal_flip=True,
                                   vertical_flip=True, 
                                   fill_mode='constant',
                                   cval = 0)    
counter2 = 0  
for ii in nifti_list:
    volume = nib.load(ii).get_data().transpose(2,0,1)[:,:,:,np.newaxis]
    volume = (255.0*(volume - volume.max())/(volume.max() - volume.min())).astype(np.uint8)
    Z,H,W,_ = volume.shape
    volume_aux = np.zeros((Z,256,256,1))
    if H > H1: H = H1
    if W > W1: W = W1
    volume_aux[:,:H,:W,0] = volume[:,:H,:W,0]
    image_generator = image_datagen.flow(volume_aux,batch_size = nslices)
    counter = 0
    for img in image_generator:
        for jj in xrange(Z):
            imsave(os.path.join(out_path,"carotid_{0:0{width}}.png".format(counter2,width = 6)),\
            img[jj,:,:,0])
            counter+=1
            counter2+=1
        if counter > nslices: break    

Number of nifti files:  50
Sampling rate per file:  1001
